# Cleaning Covid data for Mapbox: provinces

**Background**: We use Covid-19 cases data in the Philippines from data from the health department and shapefile processed through geopandas to create an interactive map. 

**Tools**: pandas, geopandas, Mapbox

As of January 21, 2023

# Do your imports

In [35]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
pd.set_option('display.max_columns', None)

# Read your CSV

In [36]:
df= pd.read_csv('provinces-cases.csv')
df

,province,cases
0,Lanao Del Sur,8300
1,Maguindanao,7788
2,Cotabato City (Not A Province),7036
3,Basilan,1879
4,Sulu,1691
5,Tawi-Tawi,774
6,NaN,20
7,Benguet,75245
8,Kalinga,12554
9,Ifugao,12225


# Cleaning the data

## Lowercase column headers

In [37]:
df.columns = df.columns.str.lower()
df.head()

,province,cases
0,Lanao Del Sur,8300
1,Maguindanao,7788
2,Cotabato City (Not A Province),7036
3,Basilan,1879
4,Sulu,1691


## Clean provincial names

This is so they would match the names in the shapefile for merging later.

In [38]:
df.province = df.province.str.replace("City Of Isabela (Not A Province)", "City of Isabela", regex=False)
df.province = df.province.str.replace("Cotabato (North Cotabato)", "Cotabato", regex=False)
df.province = df.province.str.replace("Samar (Western Samar)", "Samar", regex=False)
df.province = df.province.str.replace("Cotabato City (Not A Province)", "Cotabato City", regex=False)
df.province = df.province.str.replace("Del", "del", regex=False)
df.province = df.province.str.replace("De", "de", regex=False)
df.province = df.province.str.replace("1", "NCR, City of Manila, First District", regex=False)

## Rename columns

This is again to match the shapefile column containing the provinces' names which is 'adm2_en'. We are also renaming the column containing the Covid-19 cases tally.

In [39]:
df= df.rename(columns={"province": "adm2_en"})
df

,adm2_en,cases
0,Lanao del Sur,8300
1,Maguindanao,7788
2,Cotabato City,7036
3,Basilan,1879
4,Sulu,1691
5,Tawi-Tawi,774
6,NaN,20
7,Benguet,75245
8,Kalinga,12554
9,Ifugao,12225


# Geopandas

## Read through file

In [40]:
provinces = gpd.read_file('maps/provinces.zip')
provinces

,Shape_Leng,Shape_Area,ADM2_EN,ADM2_PCODE,ADM2_REF,ADM2ALT1EN,ADM2ALT2EN,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
0,2.640967,0.334223,Abra,PH140100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.96109 17.95348, 120.97201 17.946..."
1,3.674955,0.220065,Agusan del Norte,PH160200000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9..."
2,5.222636,0.693968,Agusan del Sur,PH160300000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((125.88961 8.98195, 125.88896 8.96446..."
3,4.626091,0.139664,Aklan,PH060400000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ..."
4,6.507665,0.205939,Albay,PH050500000,None,None,None,Region V,PH050000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ..."
5,6.045478,0.226467,Antique,PH060600000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.91984 10.42031, 121.92010 ..."
6,3.207010,0.340498,Apayao,PH148100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((121.22208 18.50058, 121.22086 18.483..."
7,6.755660,0.255767,Aurora,PH037700000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.59520 15.63249, 121.59521 ..."
8,6.925803,0.092730,Basilan,PH150700000,None,None,None,Bangsamoro Autonomous Region in Muslim Mindanao,PH150000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.99380 6.28772, 121.99383 6..."
9,2.661168,0.104455,Bataan,PH030800000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.46740 14.92276, 120.47357 14.920..."


## Merge data 

In [41]:
provinces.columns = provinces.columns.str.lower()
provinces

,shape_leng,shape_area,adm2_en,adm2_pcode,adm2_ref,adm2alt1en,adm2alt2en,adm1_en,adm1_pcode,adm0_en,adm0_pcode,date,validon,validto,geometry
0,2.640967,0.334223,Abra,PH140100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.96109 17.95348, 120.97201 17.946..."
1,3.674955,0.220065,Agusan del Norte,PH160200000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9..."
2,5.222636,0.693968,Agusan del Sur,PH160300000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((125.88961 8.98195, 125.88896 8.96446..."
3,4.626091,0.139664,Aklan,PH060400000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ..."
4,6.507665,0.205939,Albay,PH050500000,None,None,None,Region V,PH050000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ..."
5,6.045478,0.226467,Antique,PH060600000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.91984 10.42031, 121.92010 ..."
6,3.207010,0.340498,Apayao,PH148100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((121.22208 18.50058, 121.22086 18.483..."
7,6.755660,0.255767,Aurora,PH037700000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.59520 15.63249, 121.59521 ..."
8,6.925803,0.092730,Basilan,PH150700000,None,None,None,Bangsamoro Autonomous Region in Muslim Mindanao,PH150000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.99380 6.28772, 121.99383 6..."
9,2.661168,0.104455,Bataan,PH030800000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.46740 14.92276, 120.47357 14.920..."


## Rename 'Compostela Valley' to 'Davao de Oro'

The name change was by virtue of a law.

In [42]:
provinces.adm2_en = provinces.adm2_en.str.replace("Compostela Valley", "Davao de Oro", regex=False)

In [43]:
provinces_cases = provinces.merge(df, on='adm2_en')
provinces_cases

,shape_leng,shape_area,adm2_en,adm2_pcode,adm2_ref,adm2alt1en,adm2alt2en,adm1_en,adm1_pcode,adm0_en,adm0_pcode,date,validon,validto,geometry,cases
0,2.640967,0.334223,Abra,PH140100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6323
1,3.674955,0.220065,Agusan del Norte,PH160200000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23441
2,5.222636,0.693968,Agusan del Sur,PH160300000,None,None,None,Region XIII,PH160000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",17683
3,4.626091,0.139664,Aklan,PH060400000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17170
4,6.507665,0.205939,Albay,PH050500000,None,None,None,Region V,PH050000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15373
5,6.045478,0.226467,Antique,PH060600000,None,None,None,Region VI,PH060000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.91984 10.42031, 121.92010 ...",10554
6,3.207010,0.340498,Apayao,PH148100000,None,None,None,Cordillera Administrative Region,PH140000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((121.22208 18.50058, 121.22086 18.483...",9547
7,6.755660,0.255767,Aurora,PH037700000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.59520 15.63249, 121.59521 ...",4433
8,6.925803,0.092730,Basilan,PH150700000,None,None,None,Bangsamoro Autonomous Region in Muslim Mindanao,PH150000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((121.99380 6.28772, 121.99383 6...",1879
9,2.661168,0.104455,Bataan,PH030800000,None,None,None,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.46740 14.92276, 120.47357 14.920...",44544


## Drop unnecessary columns

In [44]:
provinces_cases = provinces_cases.drop(['adm2_ref', 'adm2_pcode','adm2alt1en', 'adm2alt2en', 'adm0_en', 'shape_leng', 'shape_area', 'adm0_pcode', 'adm1_pcode', 'date', 'validon', 'validto'], axis=1)
provinces_cases

,adm2_en,adm1_en,geometry,cases
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6323
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23441
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",17683
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17170
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15373
5,Antique,Region VI,"MULTIPOLYGON (((121.91984 10.42031, 121.92010 ...",10554
6,Apayao,Cordillera Administrative Region,"POLYGON ((121.22208 18.50058, 121.22086 18.483...",9547
7,Aurora,Region III,"MULTIPOLYGON (((121.59520 15.63249, 121.59521 ...",4433
8,Basilan,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((121.99380 6.28772, 121.99383 6...",1879
9,Bataan,Region III,"POLYGON ((120.46740 14.92276, 120.47357 14.920...",44544


In [45]:
provinces_cases= provinces_cases.rename(columns={"adm2_en": "province"})

## Read and merge with population data

In [46]:
df2 = pd.read_excel('population.xlsx', sheet_name="province")
df2.head()

,province,population
0,"NCR, City of Manila, First District",1846513
1,"NCR, Second District",4771371
2,"NCR, Third District",3004627
3,"NCR, Fourth District",3861951
4,Abra,250985


In [47]:
provinces_final = provinces_cases.merge(df2, on='province')
provinces_final.head()

,province,adm1_en,geometry,cases,population
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6323,250985
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23441,760413
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",17683,739367
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17170,615475
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15373,1374768


## Compute for population ratio

We do this by dividing the number of Covid-19 cases to total population per province and then multiply by 100,000. That would give us cases per 100,000 people in the area.

In [48]:
provinces_final ['case_per_pop'] = provinces_final.cases / provinces_final.population * 100000
provinces_final = provinces_final.round(1)
provinces_final.head()

,province,adm1_en,geometry,cases,population,case_per_pop
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6323,250985,2519.3
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23441,760413,3082.7
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",17683,739367,2391.6
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17170,615475,2789.7
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15373,1374768,1118.2


## Create bins for cases

The bins will allow us to categorize the number of cases, necessary for mapping later.

In [49]:
provinces_final['percentiles'] = pd.cut(np.array(provinces_final['case_per_pop']),
       [0, 1001, 2001, 3001, 4001, 5001, 6001, 7001, 8001, 9001, 10001, 11001,12000], labels=["0-1000", "1001-2000", "2001-3000", "3001-4000", "4001-5000", "5001-6000", "6001-7000", "7001-8000", "8001-9000", "9001-10000", "10001-11000", "11001-12000"])


In [50]:
provinces_final

,province,adm1_en,geometry,cases,population,case_per_pop,percentiles
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6323,250985,2519.3,2001-3000
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23441,760413,3082.7,3001-4000
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",17683,739367,2391.6,2001-3000
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17170,615475,2789.7,2001-3000
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15373,1374768,1118.2,1001-2000
5,Antique,Region VI,"MULTIPOLYGON (((121.91984 10.42031, 121.92010 ...",10554,612974,1721.8,1001-2000
6,Apayao,Cordillera Administrative Region,"POLYGON ((121.22208 18.50058, 121.22086 18.483...",9547,124366,7676.5,7001-8000
7,Aurora,Region III,"MULTIPOLYGON (((121.59520 15.63249, 121.59521 ...",4433,235750,1880.4,1001-2000
8,Basilan,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((121.99380 6.28772, 121.99383 6...",1879,426207,440.9,0-1000
9,Bataan,Region III,"POLYGON ((120.46740 14.92276, 120.47357 14.920...",44544,853373,5219.8,5001-6000


In [51]:
provinces_final.dtypes

province          object
adm1_en           object
geometry        geometry
cases              int64
population         int64
case_per_pop     float64
percentiles     category
dtype: object

**Additional step**: Convert the contents of the percentiles into string. Not doing so will not be read by the GEOJSON file.

In [52]:
provinces_final.percentiles = provinces_final.percentiles.astype(str)

In [53]:
provinces_final

,province,adm1_en,geometry,cases,population,case_per_pop,percentiles
0,Abra,Cordillera Administrative Region,"POLYGON ((120.96109 17.95348, 120.97201 17.946...",6323,250985,2519.3,2001-3000
1,Agusan del Norte,Region XIII,"MULTIPOLYGON (((125.58886 9.45793, 125.59687 9...",23441,760413,3082.7,3001-4000
2,Agusan del Sur,Region XIII,"POLYGON ((125.88961 8.98195, 125.88896 8.96446...",17683,739367,2391.6,2001-3000
3,Aklan,Region VI,"MULTIPOLYGON (((122.43980 11.59717, 122.43979 ...",17170,615475,2789.7,2001-3000
4,Albay,Region V,"MULTIPOLYGON (((124.20992 13.16871, 124.20993 ...",15373,1374768,1118.2,1001-2000
5,Antique,Region VI,"MULTIPOLYGON (((121.91984 10.42031, 121.92010 ...",10554,612974,1721.8,1001-2000
6,Apayao,Cordillera Administrative Region,"POLYGON ((121.22208 18.50058, 121.22086 18.483...",9547,124366,7676.5,7001-8000
7,Aurora,Region III,"MULTIPOLYGON (((121.59520 15.63249, 121.59521 ...",4433,235750,1880.4,1001-2000
8,Basilan,Bangsamoro Autonomous Region in Muslim Mindanao,"MULTIPOLYGON (((121.99380 6.28772, 121.99383 6...",1879,426207,440.9,0-1000
9,Bataan,Region III,"POLYGON ((120.46740 14.92276, 120.47357 14.920...",44544,853373,5219.8,5001-6000


In [54]:
provinces_final.sort_values('percentiles', ascending=False)

,province,adm1_en,geometry,cases,population,case_per_pop,percentiles
12,Benguet,Cordillera Administrative Region,"POLYGON ((120.75882 16.92650, 120.77062 16.921...",75245,827041,9098.1,9001-10000
53,"NCR, City of Manila, First District",National Capital Region,"MULTIPOLYGON (((120.94997 14.63625, 120.94997 ...",161656,1846513,8754.7,8001-9000
52,Mountain Province,Cordillera Administrative Region,"POLYGON ((121.48738 17.28923, 121.48795 17.289...",11159,158200,7053.7,7001-8000
6,Apayao,Cordillera Administrative Region,"POLYGON ((121.22208 18.50058, 121.22086 18.483...",9547,124366,7676.5,7001-8000
56,"NCR, Third District",National Capital Region,"POLYGON ((121.03842 14.78525, 121.03876 14.785...",195354,3004627,6501.8,6001-7000
43,Laguna,Region IV-A,"MULTIPOLYGON (((121.20658 14.21704, 121.20635 ...",178899,3382193,5289.4,5001-6000
9,Bataan,Region III,"POLYGON ((120.46740 14.92276, 120.47357 14.920...",44544,853373,5219.8,5001-6000
41,Kalinga,Cordillera Administrative Region,"POLYGON ((121.43840 17.69063, 121.43868 17.690...",12554,229570,5468.5,5001-6000
23,Cavite,Region IV-A,"MULTIPOLYGON (((120.61402 14.27245, 120.61404 ...",239167,4344829,5504.6,5001-6000
17,Cagayan,Region II,"MULTIPOLYGON (((122.32754 18.24594, 122.32689 ...",66057,1268603,5207.1,5001-6000


# Save as GEOJSON file

In [55]:
provinces_final.to_file('provinces_cases.geojson', driver='GeoJSON')

# Simplified file

So we were successful in combining geometry files with our dataset, but the file is too big. We, therefore, use [mapshaper](https://mapshaper.org/) to simplify the precision of the map so that we have a smaller map size.

Below is the simplified json file. 

In [56]:
simplified = gpd.read_file('provinces_cases.json')
simplified

,province,adm1_en,cases,population,case_per_pop,percentiles,geometry
0,Abra,Cordillera Administrative Region,6323,250985,2519.3,2001-3000,"POLYGON ((121.11067 17.68489, 121.10883 17.739..."
1,Agusan del Norte,Region XIII,23441,760413,3082.7,3001-4000,"MULTIPOLYGON (((125.74862 9.32604, 125.74822 9..."
2,Agusan del Sur,Region XIII,17683,739367,2391.6,2001-3000,"POLYGON ((126.22779 8.00019, 126.22786 8.00055..."
3,Aklan,Region VI,17170,615475,2789.7,2001-3000,"MULTIPOLYGON (((122.42966 11.59990, 122.42951 ..."
4,Albay,Region V,15373,1374768,1118.2,1001-2000,"MULTIPOLYGON (((124.20992 13.16871, 124.20964 ..."
5,Antique,Region VI,10554,612974,1721.8,1001-2000,"MULTIPOLYGON (((121.91984 10.42031, 121.91962 ..."
6,Apayao,Cordillera Administrative Region,9547,124366,7676.5,7001-8000,"POLYGON ((121.43655 17.69323, 121.43506 17.697..."
7,Aurora,Region III,4433,235750,1880.4,1001-2000,"MULTIPOLYGON (((121.59520 15.63249, 121.59499 ..."
8,Basilan,Bangsamoro Autonomous Region in Muslim Mindanao,1879,426207,440.9,0-1000,"MULTIPOLYGON (((121.99380 6.28772, 121.99346 6..."
9,Bataan,Region III,44544,853373,5219.8,5001-6000,"POLYGON ((120.55426 14.82074, 120.55409 14.820..."


## Convert to GEOJSON

In [57]:
simplified.to_file('simplified_provinces.geojson', driver='GeoJSON')

## Fix CSV to upload to website

In [58]:
simplified = simplified.drop(['geometry', 'percentiles', 'population'], axis=1)
simplified

,province,adm1_en,cases,case_per_pop
0,Abra,Cordillera Administrative Region,6323,2519.3
1,Agusan del Norte,Region XIII,23441,3082.7
2,Agusan del Sur,Region XIII,17683,2391.6
3,Aklan,Region VI,17170,2789.7
4,Albay,Region V,15373,1118.2
5,Antique,Region VI,10554,1721.8
6,Apayao,Cordillera Administrative Region,9547,7676.5
7,Aurora,Region III,4433,1880.4
8,Basilan,Bangsamoro Autonomous Region in Muslim Mindanao,1879,440.9
9,Bataan,Region III,44544,5219.8


In [59]:
simplified.columns = ['Province', 'Region', 'Covid-19 cases', 'Case per population']
simplified

,Province,Region,Covid-19 cases,Case per population
0,Abra,Cordillera Administrative Region,6323,2519.3
1,Agusan del Norte,Region XIII,23441,3082.7
2,Agusan del Sur,Region XIII,17683,2391.6
3,Aklan,Region VI,17170,2789.7
4,Albay,Region V,15373,1118.2
5,Antique,Region VI,10554,1721.8
6,Apayao,Cordillera Administrative Region,9547,7676.5
7,Aurora,Region III,4433,1880.4
8,Basilan,Bangsamoro Autonomous Region in Muslim Mindanao,1879,440.9
9,Bataan,Region III,44544,5219.8


In [60]:
simplified.to_csv('provinces-cases.csv', index=False)